
<img width= "300" src="https://cdn.discordapp.com/attachments/392490318798389248/945695517004951633/jeanettes_fireart.png" alt="US Fires Logo">

# Analyzing US Wildfires: Final Report

### by Lori Ainslie, Jeanette Schulz, Kristine Cabanela, and Sophia Stewart 
### Last Updated: 22 February 2022

In [1]:
# Data Science Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Group-made functions
from wrangle import *

# Blocking Warning Boxes
import warnings
warnings.filterwarnings("ignore")

# Remove Limits On Viewing Dataframes
pd.set_option('display.max_columns', None)


---
# Acquire
---

In [2]:
# code here

---
# Prepare
---

In [3]:
# code here

---
# Explore
---

In [4]:
# code here

In [ ]:
# plot counts by year
sns.countplot(data=df, x='year')

In [ ]:
# plot counts by month
sns.countplot(data=df, x='month')

In [ ]:
# plot counts by state
plt.figure(figsize=(10,20))
sns.countplot(data=df, y='state', order=df.state.value_counts().index)

In [ ]:
# plot number of fires over the years resampled by 6 months
num_fires_by_month.resample('6M').mean().plot(marker='o')

In [ ]:
# plot number of fires over the years resampled by a year
num_fires_by_month.resample('Y').mean().plot(marker='o')

In [ ]:
size_fires_by_month = df.resample('M').fire_size.sum()
size_fires_by_month.idxmax()
size_fires_by_month.max()

In [ ]:
# plot monthly summed size of fires over the years
df.groupby('year').fire_size.sum().plot()

In [ ]:
sns.lineplot(data=num_fires_by_year, x='year', y='num_fires', hue='state')

In [ ]:
# plot trends over the years by the top four causes of wildfires, excluding observations where cause was not specified
sns.lineplot(data=num_causes_by_year, x='year', y='num_occurrences', hue='general_cause')

In [ ]:
natural = num_causes_by_year[num_causes_by_year.general_cause == "Natural"]
arson = num_causes_by_year[num_causes_by_year.general_cause == "Arson/incendiarism"]
debris = num_causes_by_year[num_causes_by_year.general_cause == "Debris and open burning"]
equipment = num_causes_by_year[num_causes_by_year.general_cause == "Equipment and vehicle use"]

natural = natural.set_index("year")
arson = arson.set_index("year")
equipment = equipment.set_index("year")
debris = debris.set_index("year")

fig, ax = plt.subplots()
labels = arson.index

ax.bar(labels, arson.num_occurrences, label="Arson/incendiarism")
ax.bar(labels, natural.num_occurrences, label="Natural Causes", bottom=arson.num_occurrences)
ax.bar(labels, debris.num_occurrences, label="Debris & Open Burning", bottom=natural.num_occurrences + arson.num_occurrences)
ax.bar(labels, equipment.num_occurrences, label="Equipment & Vehicle Use", bottom=debris.num_occurrences + natural.num_occurrences + arson.num_occurrences)

ax.legend()
ax.set_ylabel("n occurrences")
ax.set_title("Top Causes Of Wildfires Across The U.S.")

#### Number of Months in a year, that had an average days_fire_existed greater than zero
- 1992: 1
- 1993: 0 
- 1994: 3
- 1995: 1
- 1996: 3
- 1997: 1
- 1998: 2
- 1999: 3
- 2000: 3
- 2001: 2
- 2002: 5
- 2003: 4
- 2004: 4
- 2005: 7
- 2006: 6
- 2007: 3
- 2008: 9
- 2009: 6
- 2010: 5
- 2011: 4
- 2012: 7
- 2013: 6
- 2014: 5
- 2015: 4
- 2016: 4
- 2017: 6
- 2018: 6

In [ ]:
# What happened in November of 2013?
clean_df[(clean_df.discovery_date == '2005-08') & (clean_df.days_fire_existed > '29 days')]

In [ ]:
# Let's look at how many fires Texas has had over the years
plt.figure(figsize = (20,8))
sns.countplot(data= texas , x= 'fire_year', hue='fire_year'  )
plt.legend([],[], frameon=False)
plt.title('Texas Wildfires have Increased in Number', fontdict={'fontsize': 24});

In [ ]:
# Total Acres burned in Texas per year
plt.figure(figsize = (20,8))
texas.groupby('fire_year').fire_size.sum().plot()
plt.title('Texas Acres Burned', fontdict={'fontsize': 24});

In [ ]:
# plotted data for cause of wildfire and # of fires

fig = plt.figure(figsize=(10, 8))
ax = fires.general_cause.value_counts().plot.bar(width=.7, ec='black', color='red')
ax.set(title='# of Fires vs cause', ylabel='Number of Fires', xlabel='cause')
plt.savefig("Dist.png")

In [ ]:
# fire size over span of years
firesizeyear=sns.barplot(x="fire_year", y="fire_size", data=fires,color='red')
firesizeyear.set_title('fire size per year')
firesizeyear.set_xticklabels(firesizeyear.get_xticklabels(),rotation=90)

In [ ]:
# plot the number of fires per day of week to visualize

plt.figure(figsize=(14,5))
g = sns.barplot(data = day_of_week, y = 'count', x = 'day_of_week')
plt.xlabel('Day of the week')
plt.ylabel('Number of wildfires')
g.axes.set_title('# of fires for day of the week',fontsize=20)

In [ ]:
# plot number of fires caused by debris and open burning over days of the week


plt.figure(figsize=(14,5))
g = sns.barplot(data = debris_over_weekday, y = 'count', x = 'day_of_week')
plt.xlabel('Day of week')
plt.ylabel('Number of wildfire cases due to Debris')
g.axes.set_title('# of wildfire per day of week caused by debris and open burning',fontsize=20)

In [ ]:
# plot fires in Texas
plt.figure(figsize=(15,15))
sns.scatterplot(x='longitude', y='latitude', data=fires[fires.state=='TX'], size='fire_size_class', hue='fire_year', palette='flare', alpha=0.5);
# light to dark, oldest to newest

In [ ]:
# look at value counts for uncontrolled windows
tx_fires.days_uncontrolled.value_counts().sort_index()

In [ ]:
# plot fires across us
plt.figure(figsize=(15,15))
sns.scatterplot(x='longitude', y='latitude', data=fires, size='fire_size', hue='fire_year', palette='flare', alpha=0.5);

In [ ]:
# print max and min for num fires/year
for year in list(range(1992, 2019)):
    print(year)
    print('Fewest fires:',fires[fires.fire_year == year].groupby('state').fire_size.count().idxmin(3), fires[fires.fire_year == year].groupby('state').fire_size.count().min())
    print('Most fires:',fires[fires.fire_year == year].groupby('state').fire_size.count().idxmax(), fires[fires.fire_year == year].groupby('state').fire_size.count().max())
    print()

---
# Conclusion
---

Summary goes here